# 基於樹的模型 (Tree-Based Models)

這個 Notebook 將介紹幾種常用的基於樹的模型，包括：

1.  **決策樹 (Decision Tree)**
    -   1.1 分類樹
    -   1.2 回歸樹
2.  **隨機森林 (Random Forest)**
3.  **梯度提升樹 (Gradient Boosting Decision Trees, GBDT)**
    -   3.1 XGBoost
    -   3.2 LightGBM

我們將使用 `pandas`、`numpy`、`matplotlib`、`seaborn`、`scikit-learn`、`xgboost` 和 `lightgbm` 等套件來實作這些模型，並使用範例資料進行說明。

## 載入需要的套件

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from sklearn.datasets import load_iris, load_boston
import xgboost as xgb
import lightgbm as lgb

---

<a id='decision-tree'></a>
## 1. 決策樹 (Decision Tree)

決策樹是一種常見的機器學習演算法，可用於**分類**和**迴歸**問題。它通過遞迴地將資料劃分為不同的子集來建構樹狀模型，每個節點代表一個特徵，每個分支代表一個決策規則，每個葉節點代表一個輸出值 (類別標籤或連續值)。

**優點:**

-   易於理解和解釋 (可視覺化)。
-   可以處理數值型和類別型特徵。
-   不需要對資料進行特徵縮放。

**缺點:**

-   容易過度擬合。
-   對訓練資料中的小變化敏感。
-   可能找到的不是全域最佳解。

**建構決策樹的關鍵概念:**

-   **特徵選擇:**  在每個節點選擇最佳的特徵來劃分資料。常用的指標包括：
    -   **資訊增益 (Information Gain)** (ID3 演算法): 基於熵 (Entropy) 的概念，選擇資訊增益最大的特徵。
    -   **增益比率 (Gain Ratio)** (C4.5 演算法): 對資訊增益的改進，考慮了特徵的取值數量。
    -   **吉尼係數 (Gini Impurity)** (CART 演算法):  衡量資料的不純度，選擇吉尼係數最小的特徵。
-   **停止條件:**  決定何時停止樹的生長，例如：
    -   達到最大深度。
    -   節點中的樣本數少於某個閾值。
    -   節點的不純度低於某個閾值。
-   **剪枝 (Pruning):**  減少樹的複雜度，防止過度擬合。常見的剪枝方法包括：
    -   **預剪枝 (Pre-pruning):**  在樹的建構過程中，提前停止樹的生長。
    -   **後剪枝 (Post-pruning):**  在樹完全生長後，再剪去一些分支。

### 1.1 分類樹

In [ ]:
# 使用 scikit-learn 載入鳶尾花資料集
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# 將資料劃分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立決策樹分類器 (使用 Gini 係數)
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42) 

# 訓練模型
clf.fit(X_train, y_train)

# 預測測試集
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# 視覺化決策樹
plt.figure(figsize=(12, 8))
plot_tree(clf, filled=True, feature_names=iris.feature_names, class_names=iris.target_names)
plt.show()

[Image of Decision Tree]

### 1.2 回歸樹

In [ ]:
# 使用 scikit-learn 載入波士頓房價資料集
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target

# 將資料劃分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立決策樹回歸器 (使用 MSE)
reg = DecisionTreeRegressor(criterion='squared_error', max_depth=3, random_state=42)

# 訓練模型
reg.fit(X_train, y_train)

# 預測測試集
y_pred = reg.predict(X_test)

# 評估模型
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R-squared (R^2):", r2_score(y_test, y_pred))

In [ ]:
# 視覺化決策樹
plt.figure(figsize=(12, 8))
plot_tree(reg, filled=True, feature_names=boston.feature_names)
plt.show()

[Image of Decision Tree Regressor]

---

<a id='random-forest'></a>
## 2. 隨機森林 (Random Forest)

隨機森林是一種**集成學習**方法，它通過建構多個決策樹並將它們的預測結果進行平均 (迴歸) 或投票 (分類) 來進行預測。隨機森林通常比單一決策樹具有更好的泛化能力，並且可以降低過度擬合的風險。

**隨機性體現在兩個方面:**

-   **樣本隨機:**  從訓練集中隨機抽取樣本 (Bootstrap 抽樣) 來訓練每個決策樹。
-   **特徵隨機:**  在每個節點分裂時，從所有特徵中隨機選擇一部分特徵來考慮。

**優點:**

-   通常具有較高的準確性。
-   可以處理大量的特徵。
-   可以估計特徵的重要性。
-   對異常值不太敏感。

**缺點:**

-   模型較為複雜，解釋性不如單一決策樹。
-   訓練時間可能較長，特別是當樹的數量較多時。

In [ ]:
# 建立隨機森林分類器
rf_clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=3, random_state=42)

# 使用鳶尾花資料集
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練模型
rf_clf.fit(X_train, y_train)

# 預測測試集
y_pred = rf_clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# 特徵重要性
feature_importances = pd.Series(rf_clf.feature_importances_, index=X.columns).sort_values(ascending=False)

# 繪製特徵重要性圖表
sns.barplot(x=feature_importances, y=feature_importances.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features (Random Forest)")
plt.show()

[Image of Feature Importances (Random Forest)]

In [ ]:
# 建立隨機森林迴歸器
rf_reg = RandomForestRegressor(n_estimators=100, criterion='squared_error', max_depth=3, random_state=42)

# 使用波士頓房價資料集
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練模型
rf_reg.fit(X_train, y_train)

# 預測測試集
y_pred = rf_reg.predict(X_test)

# 評估模型
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R-squared (R^2):", r2_score(y_test, y_pred))

---

<a id='gradient-boosting'></a>
## 3. 梯度提升樹 (Gradient Boosting Decision Trees, GBDT)

梯度提升樹 (GBDT) 也是一種**集成學習**方法，它通過**循序地**建構多個決策樹來進行預測。每個新的決策樹都試圖修正前一個決策樹的預測誤差。GBDT 通常比隨機森林具有更好的準確性，但訓練時間也更長，且更容易過度擬合。

**核心思想:**

1.  建構一個初始的決策樹 (通常是一個簡單的樹)。
2.  計算當前模型的預測誤差 (殘差)。
3.  建構一個新的決策樹來預測殘差。
4.  將新的決策樹加到現有模型中，並更新預測結果。
5.  重複步驟 2-4，直到達到指定的樹的數量或誤差不再下降。

**優點:**

-   通常具有很高的準確性。
-   可以處理數值型和類別型特徵。
-   可以估計特徵的重要性。

**缺點:**

-   訓練時間可能較長，特別是當樹的數量較多時。
-   容易過度擬合，需要仔細調整參數。
-   模型較為複雜，解釋性不如單一決策樹。

### 3.1 XGBoost

XGBoost (Extreme Gradient Boosting) 是一個高效且流行的 GBDT 實作，它在原始 GBDT 演算法的基礎上進行了多項改進，包括：

-   **正則化:**  XGBoost 在目標函數中加入了正則化項，以控制模型的複雜度，防止過度擬合。
-   **二階導數:** XGBoost 使用了損失函數的二階導數 (Hessian 矩陣) 來加速訓練過程，並提高模型的準確性。
-   **缺失值處理:** XGBoost 可以自動處理缺失值。
-   **平行化:** XGBoost 支援平行化訓練，可以加快訓練速度。
-   **高效能:** XGBoost 經過了優化，具有很高的執行效率。

首先，您需要安裝 XGBoost 套件：

```bash
pip install xgboost
```

In [ ]:
# 建立 XGBoost 分類器
xgb_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# 使用鳶尾花資料集
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練模型
xgb_clf.fit(X_train, y_train)

# 預測測試集
y_pred = xgb_clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# 特徵重要性
feature_importances = pd.Series(xgb_clf.feature_importances_, index=X.columns).sort_values(ascending=False)

# 繪製特徵重要性圖表
sns.barplot(x=feature_importances, y=feature_importances.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features (XGBoost)")
plt.show()

[Image of Feature Importances (XGBoost)]

In [ ]:
# 建立 XGBoost 迴歸器
xgb_reg = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# 使用波士頓房價資料集
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練模型
xgb_reg.fit(X_train, y_train)

# 預測測試集
y_pred = xgb_reg.predict(X_test)

# 評估模型
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R-squared (R^2):", r2_score(y_test, y_pred))

### 3.2 LightGBM

LightGBM 是一個由微軟開發的 GBDT 框架，它在 XGBoost 的基礎上進行了進一步的優化，主要包括：

-   **基於直方圖的演算法:** LightGBM 使用基於直方圖的演算法來尋找最佳的節點分裂點，可以減少記憶體使用量和計算時間。
-   **帶有深度限制的葉子生長 (Leaf-wise) 策略:**  與 XGBoost 的按層生長 (Level-wise) 策略不同，LightGBM 採用帶有深度限制的葉子生長策略，可以更快地收斂，並降低過度擬合的風險。
-   **特徵並行和資料並行:** LightGBM 支援特徵並行和資料並行，可以進一步加快訓練速度。
-   **高效能:** LightGBM 通常比 XGBoost 訓練速度更快，記憶體使用量更低。

首先，您需要安裝 LightGBM 套件：

```bash
pip install lightgbm
```

In [ ]:
# 建立 LightGBM 分類器
lgb_clf = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# 使用鳶尾花資料集
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練模型
lgb_clf.fit(X_train, y_train)

# 預測測試集
y_pred = lgb_clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# 特徵重要性
feature_importances = pd.Series(lgb_clf.feature_importances_, index=X.columns).sort_values(ascending=False)

# 繪製特徵重要性圖表
sns.barplot(x=feature_importances, y=feature_importances.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features (LightGBM)")
plt.show()

[Image of Feature Importances (LightGBM)]

In [ ]:
# 建立 LightGBM 迴歸器
lgb_reg = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# 使用波士頓房價資料集
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練模型
lgb_reg.fit(X_train, y_train)

# 預測測試集
y_pred = lgb_reg.predict(X_test)

# 評估模型
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R-squared (R^2):", r2_score(y_test, y_pred))

---

## 總結

這個 Notebook 介紹了幾種常用的基於樹的模型，包括決策樹、隨機森林和梯度提升樹 (XGBoost、LightGBM)。這些模型在各種機器學習任務中都表現出色，是您機器學習工具箱中不可或缺的一部分。

**選擇哪個模型?**

-   **決策樹:**  簡單易懂，可作為 baseline 模型，或用於需要模型解釋性的場景。
-   **隨機森林:**  通常比決策樹具有更好的準確性和泛化能力，適用於大多數情況。
-   **GBDT (XGBoost、LightGBM):**  通常具有最高的準確性，但需要更長的訓練時間和更多的參數調整。XGBoost 更為成熟穩定，LightGBM 則通常更快更輕量。

建議您根據具體的應用場景和資料特性來選擇合適的模型，並通過交叉驗證等方法來評估模型的效能，並調整模型的參數以獲得最佳結果。

## 參考資料

-   [Scikit-learn Documentation](https://scikit-learn.org/stable/)
-   [XGBoost Documentation](https://xgboost.readthedocs.io/)
-   [LightGBM Documentation](https://lightgbm.readthedocs.io/)